In [1]:
import pandas as pd
from docx import Document
import os
from itertools import permutations
import re

In [2]:
def abrir_docx(fname, word_dir = 'original_data/word/'):
    
    file = os.path.join(word_dir, fname)
    
    return Document(file)

In [3]:
def cell_text(table, i, j):
    
    return table.cell(i, j).text

In [4]:
def pegar_valores(table, dict_dados):
    
    return {chave : cell_text(table, val[0], val[1]) for chave, val in dict_dados.items()}

In [5]:
def ancorar(table, texto, ini, fim = 100, col =0):
        
    for i in range(ini, fim):
        if cell_text(table, i, col) == texto:
            return i
    

In [6]:
def pegar_meta(table):
    
    dados = {
        'meta_numero' : (0,0),
        'desc_meta' : (0, 2)
    }
    
    return pegar_valores(table, dados)

In [7]:
def pegar_pdm(table):
    
    ancora_ini = ancorar(table, 'Vinculação com o Programa de Metas 2021-2024', 0) + 1
    ancora_fim = ancorar(table,'Vinculação com o Plano Plurianual 2022-2025', 0) - 1
    
    dados = []
    schema = {'meta_pdm_num' : None,
             'meta_pdm_desc' : None}

    for i in range(ancora_ini, ancora_fim):
        
        schema['meta_pdm_num'] = (i, 0)
        schema['meta_pdm_desc'] = (i, 2)
        
        dados.append(pegar_valores(table, schema))
    
    return dados

In [8]:
def pegar_ppa_raw(table):
    
    ancora_ini = ancorar(table, 'Vinculação com o Plano Plurianual 2022-2025', 0) + 1
    ancora_fim = ancorar(table,'Indicadores da Agenda Municipal 2030 priorizados', 0) - 1
    
    dados = []
    schema = {'key' : None,
             'val' : None}
    n = 0
    for i in range(ancora_ini, ancora_fim):
        
        schema['key'] = (i, 0)
        schema['val'] = (i, 2)
        
        buscado = pegar_valores(table, schema)
        dados.append(buscado)
    
    return dados

In [9]:
def parsear_ppa(dados_ppa_raw):
    
    dados = []
    parseado = {}
    for item in dados_ppa_raw:
        if 'objetivo est' not in item['key'].lower():
            parseado['programa_num'] = item['key']
            parseado['programa_desc'] = item['val']
        else:
            parseado['objetivo_estrategico'] = item['val']
            dados.append(parseado)
            parseado = {}
    return dados

In [10]:
def pegar_ppa(table):
    
    dados_ppa_raw = pegar_ppa_raw(table)
    parsed = parsear_ppa(dados_ppa_raw)
    
    return parsed

In [11]:
def pegar_indicadores_agenda(table):
    
    ancora_ini = ancorar(table, 'Número', 0) + 1
    ancora_fim = ancorar(table,'Ação', 0) - 1
    
    dados = []
    schema = {'numero' : None,
             'redacao' : None,
             'meta_2030' : None,
             'previsao_2024' : None}
    n = 0
    for i in range(ancora_ini, ancora_fim):
        
        schema['numero'] = (i, 0)
        schema['redacao'] = (i, 1)
        schema['meta_2030'] = (i, 2)
        schema['previsao_2024'] = (i, 3)
        
        buscado = pegar_valores(table, schema)
        dados.append(buscado)
    
    return dados

In [12]:
def pegar_acoes(table):
    
    ancora_ini = ancorar(table, 'Ação', 0) + 1
    ancora_fim = len(table.rows)
    
    dados = []
    schema = {'letra_acao' : None,
             'desc_acao' : None,
             'orgao_resp' : None,
             'marco' : None}
    n = 0
    for i in range(ancora_ini, ancora_fim):
        
        schema['letra_acao'] = (i, 0)
        schema['desc_acao'] = (i, 1)
        schema['orgao_resp'] = (i, 2)
        schema['marco'] = (i, 3)
        
        buscado = pegar_valores(table, schema)
        dados.append(buscado)
    
    return dados

In [13]:
def generate_table(doc, func):
    
    dados = []
    for table in doc.tables[1:]:
        meta = pegar_meta(table)
        dados_func = func(table)
        if type(dados_func) is dict:
            meta.update(dados_func)
            dados.append(meta)
        else:
            for item in dados_func:
                item.update(meta)
                dados.append(item)
        
    return pd.DataFrame(dados)

In [14]:
doc = abrir_docx('FINAL Fichas Plano de Ação ODS.docx')

In [15]:
pdm = generate_table(doc, pegar_pdm)


In [16]:
pdm.sample(3)

,meta_pdm_num,meta_pdm_desc,meta_numero,desc_meta
2,Meta 24,"Atingir o resultado de 5,2 no IDEP para os ano...",4.1,"Até 2030, garantir que todas as meninas e meni..."
36,Meta 57,"Realizar 600.000 atendimentos ao trabalhador, ...",8.6,"Até 2030, reduzir substancialmente a proporção..."
75,Sem vinculação,,8.7,Tomar medidas imediatas e eficazes para erradi...


In [17]:
pdm.fillna('sem vinculação', inplace=True)

In [18]:
ppa = generate_table(doc, pegar_ppa)

In [19]:
ppa.sample(3)

,programa_num,programa_desc,objetivo_estrategico,meta_numero,desc_meta
18,3015,PROMOÇÃO DA CIDADE COMO CENTRO DE EVENTOS E RE...,Promover a cooperação internacional e posicion...,12.a,Formalizar parcerias nacionais e internacionai...
122,3013,"PROMOÇÃO DA SEGURANÇA URBANA, PREVENÇÃO E PROT...",Ampliar a capacidade de monitoramento em segur...,15.3,"Até 2030, reduzir a degradação do solo no muni..."
78,Programa 3023,PROTEÇÃO À POPULAÇÃO EM SITUAÇÃO DE VULNERABIL...,Reduzir a pobreza e ampliar o acesso a direito...,8.7,Tomar medidas imediatas e eficazes para erradi...


In [20]:
indi = generate_table(doc, pegar_indicadores_agenda)

In [21]:
indi.sample(3)

,numero,redacao,meta_2030,previsao_2024,meta_numero,desc_meta
201,15.5.1,Animais silvestres reabilitados para soltura n...,Monitorar,1000/ano,15.5,"Proteger a fauna silvestre do município, por m..."
186,Sem indicadores priorizados,,,,13.3,"Até 2030, promover e aprofundar as temáticas d..."
9,sem indicadores priorizados,,,,4.c,"Até 2030, substancialmente aumentar o continge..."


In [22]:
indi.fillna('Sem indicadores priorizados', inplace=True)

In [23]:
acoes = generate_table(doc, pegar_acoes)

In [24]:
acoes.sample(3)

,letra_acao,desc_acao,orgao_resp,marco,meta_numero,desc_meta
469,D,Apoiar tecnicamente as UVIS e as CRS para apri...,DVE/COVISA/SMS,Ações de rotina.,3.d,Reforçar a capacidade local para o alerta prec...
395,C,Ofertar capacitações nos laboratórios de fabri...,SMIT,Número de pessoas capacitadas,5.b,Garantir a igualdade de gênero no acesso a hab...
66,S,Realizar atividades formativas de debate e enf...,SVMA,Atividades formativas de debate e enfrentament...,4.7,"Até 2030, ampliar o acesso a conhecimentos e h..."


In [25]:
writer = pd.ExcelWriter('dados_extraidos_agenda_2030.xlsx', engine='xlsxwriter')

In [26]:
pdm.to_excel(writer, sheet_name='vinculacao_pdm')
ppa.to_excel(writer, sheet_name='vinculacao_ppa')
indi.to_excel(writer, sheet_name='indicadores_priorizados')
acoes.to_excel(writer, sheet_name='acoes')

In [27]:
writer.save()